In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Komoran
import re
from dateutil import parser

def extract_schedule_info(sentence):
    # KoNLPy 형태소 분석기 초기화
    komoran = Komoran()

    # dateutil.parser를 사용하여 날짜 및 시간 파싱
    try:
        date_time = parser.parse(sentence, fuzzy=True)
        date_time_str = date_time.strftime("%Y-%m-%d %H:%M:%S")
    except ValueError:
        date_time_str = "날짜 및 시간을 찾을 수 없음"

    # KoNLPy를 사용하여 명사 추출
    nouns = komoran.nouns(sentence)

    # "내일", "오전", "오후", "시" 및 날짜 전용 명사들을 제외한 명사로 이루어진 일정 정보 추출
    excluded_words = ["내일", "오전", "오후", "시", "이번", "다음", "이번 주", "다음 주", "다음 달", "이번 달", "올해", "내년", "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "몇월", "몇일", "몇시"]
    event_words = [word for word in nouns if word not in excluded_words and not re.match(r'\d+[월일시]', word)]

    # "에" 다음에 오는 명사로 이루어진 일정 정보 추출
    event_match = re.search(r'(?<=에 ).*?(?<![\w\d가-힣])\.$', sentence)
    if event_match and not date_time:
        event_words.extend([word for word in event_match.group().split() if word not in excluded_words and not re.match(r'\d+[월일시]', word)])

    # 주기적인 일정 처리
    # "매주" 또는 "매월"이 포함되어 있으면 해당 단어 제외
    event_words = [word for word in event_words if word not in ["매주", "매월"]]

    # "다음 달"이라는 표현이 나오면 "달"을 명사에서 제외
    if "다음 달" in sentence:
        event_words.remove("달")

    # "일"이라는 표현이 "마감" 뒤에 오면 "일"을 명사에서 제외
    if "일" in event_words and "마감" in event_words:
        event_words.remove("일")

    # Exclude the word following a number if "일" immediately follows
    for i in range(len(event_words) - 1):
        if re.match(r'\d+', event_words[i]) and event_words[i + 1] == "일":
            event_words.pop(i + 1)

    # Remove leading "일" only if it appears at the beginning
    if event_words and event_words[0] == "일":
        event_words = event_words[1:]

    event = " ".join(event_words) if event_words else "todo"

    return date_time_str, event

# 주어진 예시 문장들
sentences = [
    "내일 오후 3시에 프로젝트 회의, 회의실 B에서 진행합니다.",
    "다음 주 월요일부터 금요일까지, 오전 10시부터 오후 4시까지 근무해요.",
    "5월 1일부터 7일까지 해외 출장 가",
    "매주 수요일 오후 2시에 팀 미팅, 진행 상황 및 이슈 논의해!.",
    "이번 주 토요일, 오후 7시에 친구 결혼식 참석 예정.",
    "다음 주 화요일 10시에 민선이랑 밥약하기로 했어.",
    "다음 주 수요일 19시에 경제학개론 과제 제출해야돼",
    "다음 주 수요일 오후 4시에 고객과의 미팅 예정.",
    "이번 주 토요일 오전 10시에 새로운 프로젝트 팀 미팅이 있습니다.",
    "7월 15일까지 서류 제출 마감일",
    "내일 오후 2시에 도서관에서 책을 반납해야 합니다.",
    "다음 달 20일까지 연차 신청해야한다.",
    "매주 화요일 오후 3시에 회의가 있습니다.",
    "매월 3일에 업무 보고가 있습니다.",
    "다음 달 20일에 연차 신청 마감이에요.",
    "월요일 3시까지 선형대수 복습",
    "다음주 화요일에 숙명여대 명신관에서 강의를 하기로 했어."
]

# 문장에서 일정 정보 추출
for sentence in sentences:
    date_time, event = extract_schedule_info(sentence)
    print(f"원문: {sentence}")
    print(f"일정 텍스트: {event}")
    print("-" * 50)
